In [1]:
import besca as bc
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
from scipy import sparse, io
import scipy.sparse as sp
import os
import re
import time
import logging
import seaborn as sns
import h5py
sc.logging.print_header()

# for standard processing, set verbosity to minimum
sc.settings.verbosity = 0  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.set_figure_params(dpi=80)
version = '2.8'
start0 = time.time()

%matplotlib inline

/home/misaell2/.conda/envs/besca/lib/python3.9/site-packages/besca/datasets/_datasets.py:8: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


scanpy==1.10.3 anndata==0.10.8 umap==0.5.9.post2 numpy==1.26.4 scipy==1.13.1 pandas==2.3.3 scikit-learn==1.6.1 statsmodels==0.14.5 igraph==1.0.0 leidenalg==0.11.0 pynndescent==0.5.13


/home/misaell2/.conda/envs/besca/lib/python3.9/site-packages/umap/__init__.py:9: ImportWarning:

Tensorflow not installed; ParametricUMAP will be unavailable

/home/misaell2/.conda/envs/besca/lib/python3.9/site-packages/scanpy/_settings.py:488: DeprecationWarning:

`set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`



In [2]:
#define standardized filepaths based on above input
root_path = os.getcwd()
bescapath_full = os.path.dirname(bc.__file__)
bescapath = os.path.split(bescapath_full)[0]

analysis_name = 'all_merged_bbknn'

# Choose the clustering to base the annotation on.
# leiden (RNA), CLR (CITE-seq), DSB (CITE-seq)
clusters='leiden' 

batch_to_correct = 'ID'

results_folder = os.path.join(root_path, 'analyzed', analysis_name)

# Input h5ad file
input_annotated = os.path.join(root_path, 'annotated.all_merged.h5ad')
input_raw = os.path.join(root_path, 'raw.all_merged.h5ad')

# Output h5ad file
results_file = os.path.join(results_folder, analysis_name + '.annotated.clustered.h5ad')

#define output directories
outdir_figures = figdir = os.path.join(results_folder, 'figures')
outdir_results = os.path.join(results_folder, 'results')
outdir_export_data = os.path.join(results_folder, 'outfiles')

sc.settings.figdir = os.path.join(outdir_figures)

In [3]:
if not os.path.exists(outdir_figures):
    os.makedirs(outdir_figures)
if not os.path.exists(outdir_results):
    os.makedirs(outdir_results)
if not os.path.exists(outdir_export_data):
    os.makedirs(outdir_export_data)

In [4]:

# Read annotated data
adata_annotated = sc.read(input_annotated)
adata_annotated

AnnData object with n_obs × n_vars = 322323 × 577
    obs: 'CELL', 'CONDITION', 'BARCODE', 'ID', 'Donor', 'celltype0', 'celltype1', 'celltype2', 'dblabel', 'technology', 'tissue'
    var: 'SYMBOL', 'ENSEMBL'
    obsm: 'X_pca', 'X_umap'

In [5]:
# Read raw data
adata_raw = sc.read(input_raw, citeseq = None)
adata_raw

AnnData object with n_obs × n_vars = 753435 × 14966
    obs: 'CELL', 'CONDITION', 'BARCODE', 'ID', 'Donor', 'technology', 'tissue'
    var: 'SYMBOL', 'ENSEMBL'

In [6]:
adata = adata_raw[adata_annotated.obs_names].copy()
adata.obs = adata_annotated.obs.copy()
adata

AnnData object with n_obs × n_vars = 322323 × 14966
    obs: 'CELL', 'CONDITION', 'BARCODE', 'ID', 'Donor', 'celltype0', 'celltype1', 'celltype2', 'dblabel', 'technology', 'tissue'
    var: 'SYMBOL', 'ENSEMBL'

In [7]:
set(adata.obs['celltype2'])

adata.obs['ID'] = adata.obs['ID'].cat.reorder_categories([
    'Donor1_PBMC_10Xfixed', 
    'Donor2_PBMC_10Xfixed', 
    'Donor3_PBMC_10Xfixed',
    'Donor1_RBCdepl_10Xfixed', 
    'Donor2_RBCdepl_10Xfixed',
    'Donor3_RBCdepl_10Xfixed', 
    'Donor1_PBMC_10X3prime',
    'Donor2_PBMC_10X3prime', 
    'Donor3_PBMC_10X3prime',
    'Donor1_RBCdepl_10X3prime', 
    'Donor2_RBCdepl_10X3prime',
    'Donor3_RBCdepl_10X3prime', 
    'Donor1_RBCdepl_Parse',
    'Donor3_RBCdepl_Parse', 
    'Donor2_RBCdepl_Parse', 
    'Donor1_PBMC_Parse', 
    'Donor2_PBMC_Parse',
    'Donor3_PBMC_Parse',
    'Donor1_WB_D0_HIVE',
    'Donor2_WB_D0_HIVE',
    'Donor3_WB_D0_HIVE',
])
adata.X.max()

14577.0

In [8]:

adata = bc.st.clustering(adata, results_folder, myres=1.0)
adata

/home/misaell2/.conda/envs/besca/lib/python3.9/site-packages/besca/st/_wrapper_funcs.py:533: FutureWarning:

In the future, the default backend for leiden will be igraph instead of leidenalg.

 To achieve the future defaults please pass: flavor="igraph" and n_iterations=2.  directed must also be False to work with igraph's implementation.



KeyError: 'No "neighbors" in .uns'

In [ ]:
import os, h5py, numpy as np, pandas as pd, scanpy as sc
import scipy.sparse as sp
from scipy.sparse import issparse

# --- Paths (adjust if needed) ---
input_annotated = os.path.join(os.getcwd(), 'annotated.all_merged.h5ad')
input_raw       = os.path.join(os.getcwd(), 'raw.all_merged.h5ad')
out_small       = 'raw.RBCsubset.h5ad'

# 1) Load annotated (small-ish)
adata_annot = sc.read_h5ad(input_annotated)

# 2) RBC mask from annotated metadata
rbc_mask = adata_annot.obs['ID'].astype(str).str.contains('RBC', case=False, na=False)
rbc_indices = np.flatnonzero(rbc_mask.values)
print(f"RBC cells: {rbc_indices.size:,}")

# 3) Open raw .h5ad and stream only RBC rows from a CSR matrix
with h5py.File(input_raw, "r") as f:
    # --- get var names and shapes (using backed AnnData for reliability)
    adata_raw_backed = sc.read_h5ad(input_raw, backed="r")
    var_names = adata_raw_backed.var_names.copy()
    n_vars    = var_names.size
    n_obs     = adata_raw_backed.n_obs
    adata_raw_backed.file.close()   # important: close manually

    # --- locate CSR storage for X
    # anndata stores CSR arrays at: X/data, X/indices, X/indptr
    Xgrp    = f['X']
    indptr  = Xgrp['indptr']   # shape: (n_obs + 1,)
    indices = Xgrp['indices']
    data    = Xgrp['data']

    # --- build CSR for the RBC subset by concatenating selected rows
    out_data    = []
    out_indices = []
    out_indptr  = [0]

    rbc_indices_sorted = np.sort(rbc_indices)
    for i in rbc_indices_sorted:
        start = int(indptr[i])
        end   = int(indptr[i + 1])
        if end > start:
            out_data.append(   data[start:end]   )
            out_indices.append(indices[start:end])
        out_indptr.append(out_indptr[-1] + (end - start))

    if out_data:
        out_data    = np.concatenate(out_data)
        out_indices = np.concatenate(out_indices)
    else:
        out_data    = np.array([], dtype=data.dtype)
        out_indices = np.array([], dtype=indices.dtype)

# 4) Construct smaller AnnData for RBC cells
X_rbc = sp.csr_matrix((out_data, out_indices, np.array(out_indptr, dtype=np.int64)),
                      shape=(rbc_indices.size, n_vars))

obs_rbc = adata_annot.obs.iloc[rbc_indices].copy()
var_df  = pd.DataFrame(index=var_names)  # minimal var frame

adata_rbc = sc.AnnData(X=X_rbc, obs=obs_rbc, var=var_df)

# 5) Precompute QC columns on the small object (sparse-safe)
def row_sum(mat):
    return np.asarray(mat.sum(axis=1)).ravel() if issparse(mat) else mat.sum(axis=1)

def row_nnz(mat):
    if issparse(mat):
        # (mat > 0) is OK for CSR; avoid densifying
        return np.asarray((mat > 0).sum(axis=1)).ravel()
    return (mat > 0).sum(axis=1)

# total UMI and detected genes
umi_counts  = row_sum(adata_rbc.X).astype(np.float64)
gene_counts = row_nnz(adata_rbc.X).astype(np.int64)
adata_rbc.obs['total_counts']       = umi_counts
adata_rbc.obs['n_genes_by_counts']  = gene_counts

# mitochondrial fraction (handles MT- or MT_)
genes  = np.asarray(adata_rbc.var_names, dtype=str)
upper  = np.char.upper(genes)
mt_mask = np.char.startswith(upper, 'MT-') | np.char.startswith(upper, 'MT_')

if mt_mask.any():
    mt_counts = row_sum(adata_rbc.X[:, mt_mask]).astype(np.float64)
else:
    mt_counts = np.zeros(adata_rbc.n_obs, dtype=np.float64)

umi_safe = umi_counts.copy()
umi_safe[umi_safe == 0] = np.nan
adata_rbc.obs['mito_fraction'] = mt_counts / umi_safe

# 6) Save the manageable file for downstream plotting
adata_rbc.write(out_small, compression='gzip')
print(f"Wrote {out_small} with shape {adata_rbc.shape}")
print(adata_rbc.obs[['total_counts','n_genes_by_counts','mito_fraction']].describe().T)


In [ ]:
rbc_cells = sc.read_h5ad("raw.RBCsubset.h5ad")
print(rbc_cells)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# map variables to right columns
tech_map = {
    '10X3prime': 'Chromium Single Cell 3′ Gene Expression v3.1 (10X Genomics)',
    '10Xfixed':  'Chromium Single Cell Gene Expression Flex (10X Genomics)',
    'Parse':     'Evercode WT Mini v2 (Parse Biosciences)',
    'HIVE':      'HIVE scRNAseq v1 (Honeycomb Biotechnologies)',
}


# Prepare tidy DF
df_qc = pd.DataFrame({
    "Donor": rbc_cells.obs["Donor"].astype(str),
    "Technology_code": rbc_cells.obs["technology"].astype(str),
    "UMI counts": rbc_cells.obs["total_counts"].values,
    "Mitochondrial fraction": rbc_cells.obs["mito_fraction"].values,
})

# Order the technologies and donors
tech_order = [t for t in ["10X3prime", "10Xfixed", "Parse", "HIVE"]
              if t in df_qc["Technology_code"].unique()]

donor_order = sorted(df_qc["Donor"].unique().tolist())

# Color map (tech → color)
palette = sns.color_palette("Set2", n_colors=len(tech_order))
tech_color_map = {tech_order[i]: palette[i] for i in range(len(tech_order))}

# Faceted violin plot: one column per technology
g = sns.catplot(
    data=df_qc,
    kind="violin",
    col="Technology_code",
    x="Donor",
    y="Mitochondrial fraction",
    order=donor_order,
    col_order=tech_order,
    hue="Technology_code",
    palette=tech_color_map,
    inner="quartile",
    cut=0,
    sharey=True,
    height=4,
    aspect=1.1
)

# y-range to match the paper
g.set(ylim=(0, 0.25))

# Replace top titles with full descriptive labels
for ax, tech in zip(g.axes.flatten(), tech_order):
    ax.set_title(tech_map.get(tech, tech))

# Clean x-axis
g.set_xlabels("Donor")
g.set_ylabels("Mitochondrial fraction")

# Remove the legend (color is redundant per panel)
g._legend.remove()

plt.suptitle("Figure 2A: Mitochondrial fraction per Donor within each Technology", y=1.05)
plt.tight_layout()
plt.show()


In [ ]:
g = sns.catplot(
    data=df_qc,
    kind="violin",
    col="Technology_code",
    x="Donor",
    y="UMI counts",
    order=donor_order,
    col_order=tech_order,
    hue="Technology_code",
    palette=tech_color_map,
    inner="quartile",
    cut=0,
    sharey=True,
    height=4,
    aspect=1.1
)

g.set(ylim=(0, 25000))

# Replace top titles
for ax, tech in zip(g.axes.flatten(), tech_order):
    ax.set_title(tech_map.get(tech, tech))

g.set_xlabels("Donor")
g.set_ylabels("UMI counts")

# Remove legend — redundant because each panel is a single tech
g._legend.remove()

plt.suptitle("Figure 2B: UMI counts per Donor within each Technology", y=1.05)
plt.tight_layout()
plt.show()


In [ ]:
# =============================================================================
# Figure 2C — “Number of genes” per Donor × Technology (RBC-depleted cells)
# =============================================================================
# Goal:
#   • Reproduce Figure 2C from the paper, using the *n_genes_by_counts*
#     QC metric as the y-axis.
#   • Each panel (column) represents a Technology (10X 3′, 10X Flex, Parse, HIVE).
#   • Within each technology, the x-axis shows Donor 1–3 side by side.
#   • Violin colors correspond to technology; all violins share the same y-range.
#
# Inputs:
#   rbc_cells.obs['Donor']               – donor/sample ID per cell
#   rbc_cells.obs['technology']          – short technology codes (10X3prime, 10Xfixed, Parse, HIVE)
#   rbc_cells.obs['n_genes_by_counts']   – number of detected genes per cell
#
# Expected output:
#   A multi-panel figure (one column per technology), each showing three violins
#   (Donor 1–3) colored by technology, y-axis from 0 → 5000.

# -------------------------------------------------------------------------
# 1) Build a tidy DataFrame for plotting
# -------------------------------------------------------------------------
df_qc_genes = pd.DataFrame({
    "Donor": rbc_cells.obs["Donor"].astype(str),
    "Technology_code": rbc_cells.obs["technology"].astype(str),
    "Number of genes": rbc_cells.obs["n_genes_by_counts"].astype(float),
})

# Order technologies (only those present in the current dataset)
tech_order = [t for t in ["10X3prime", "10Xfixed", "Parse", "HIVE"]
              if t in df_qc_genes["Technology_code"].unique()]

# Order donors alphabetically/numerically for consistent x-axis placement
donor_order = sorted(df_qc_genes["Donor"].unique().tolist())

# -------------------------------------------------------------------------
# 2) Define color palette (consistent with earlier figures)
# -------------------------------------------------------------------------
# We assign one distinct color per technology for visual consistency
base_palette = sns.color_palette("Set2", n_colors=4)
tech_color_map = {code: base_palette[i] for i, code in enumerate(["10X3prime", "10Xfixed", "Parse", "HIVE"])}
palette_list = [tech_color_map[c] for c in tech_order]

# -------------------------------------------------------------------------
# 3) Plot — one panel per technology, violins grouped by donor
# -------------------------------------------------------------------------
g = sns.catplot(
    data=df_qc_genes,
    kind="violin",
    col="Technology_code",        # → each column = one technology
    x="Donor",                    # → donors on x-axis within each technology
    y="Number of genes",          # → QC metric on y-axis
    order=donor_order,            # consistent donor order
    col_order=tech_order,         # consistent tech order (left→right)
    hue="Technology_code",        # color violins by technology
    palette=tech_color_map,       # fixed color map (ensures consistency)
    inner="quartile",             # show median + quartile lines inside violins
    cut=0,                        # don't extend violins beyond data range
    sharey=True,                  # same y-axis scale across panels
    height=4,                     # panel height (in inches)
    aspect=1.1                    # width/height ratio
)

# -------------------------------------------------------------------------
# 4) Styling / axes / titles
# -------------------------------------------------------------------------
# Set a consistent y-axis range to match the paper
g.set(ylim=(0, 5000))

# Replace the short technology codes in the titles with descriptive names
tech_map = {
    "10X3prime": "Chromium Single Cell 3′ Gene Expression v3.1 (10X Genomics)",
    "10Xfixed":  "Chromium Single Cell Gene Expression Flex (10X Genomics)",
    "Parse":     "Evercode WT Mini v2 (Parse Biosciences)",
    "HIVE":      "HIVE scRNAseq v1 (Honeycomb Biotechnologies)",
}
for ax, tech in zip(g.axes.flatten(), tech_order):
    ax.set_title(tech_map.get(tech, tech))

# Label axes cleanly
g.set_xlabels("Donor")
g.set_ylabels("Number of genes detected per cell")

# Remove legend (redundant, color = panel)
if g._legend is not None:
    g._legend.remove()

# Global figure title and layout
plt.suptitle("Figure 2C – Number of genes per Donor within each Technology (RBC-depleted)", y=1.05)
plt.tight_layout()
plt.show()

# -------------------------------------------------------------------------
# 5) Sanity checks / interpretation
# -------------------------------------------------------------------------
# • Each technology panel shows 3 violins (Donor 1–3).
# • The y-axis range (0–5000) highlights that typical single-cell libraries
#   detect hundreds to a few thousand genes per cell.
# • Expect Parse and 10X Flex to cluster near 1–3 k genes; 10X 3′ may extend higher.
# • If a panel is missing → that technology had no RBC-depleted cells in your dataset.


In [ ]:
# =============================================================================
# Figure 2D (extended) — Dot plot across Technologies *and* Cell Types
# =============================================================================
# Goal:
#   • Rows = first each Technology, then key immune Cell Types (B/T/Mono/NK/Neutrophil).
#   • Columns = selected genes (stable markers).
#   • Dot size  = fraction of cells in the group expressing the gene (>0 counts).
#   • Dot color = log1p(mean raw expression) in the group (white→dark red).
#
# Inputs (from previous steps):
#   rbc_cells: AnnData with raw counts in .X and metadata in .obs
#   rbc_cells.obs['technology']: technology code ('10X3prime','10Xfixed','Parse','HIVE')
#   rbc_cells.obs['celltype*'] : cell type labels (we'll auto-pick a suitable column)
#   rbc_cells.var_names: gene symbols
#
# Output:
#   A composite dot plot with Technologies on top followed by selected Cell Types;
#   right side shows a size legend for fraction (%) and a colorbar for mean expression.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse import issparse

# -----------------------------
# 1) Configure genes & columns
# -----------------------------
genes_for_dotplot = [
    "S100A8", "S100A9", "MPO", "ELANE", "LTF", "CEACAM8", "CXCR2",
    "CSF3R", "FCGR3B", "OLFM4", "LYZ", "GAPDH"
]
present_genes = [g for g in genes_for_dotplot if g in rbc_cells.var_names]
if not present_genes:
    raise ValueError("None of the selected genes are present in rbc_cells.var_names.")

# Pick a suitable cell-type column
celltype_candidates = ["celltype1", "celltype0", "celltype2", "cell_type", "CellType"]
celltype_col = next((c for c in celltype_candidates if c in rbc_cells.obs.columns), None)
if celltype_col is None:
    raise KeyError("Could not find a cell-type column in rbc_cells.obs.")

# Cell types to include (in the paper’s order)
wanted_celltypes = ["B cell", "T cell", "monocyte", "natural killer cell", "neutrophil"]

# Technologies (order for rows)
tech_codes = rbc_cells.obs["technology"].astype(str).unique().tolist()
tech_order = [t for t in ["10X3prime", "10Xfixed", "Parse", "HIVE"] if t in tech_codes]

# Pretty mapping for technology labels (used in y-tick text)
tech_map = {
    "10X3prime": "Chromium Single Cell 3′ Gene Expression v3.1 (10X Genomics)",
    "10Xfixed":  "Chromium Single Cell Gene Expression Flex (10X Genomics)",
    "Parse":     "Evercode WT Mini v2 (Parse Biosciences)",
    "HIVE":      "HIVE scRNAseq v1 (Honeycomb Biotechnologies)",
}

# Helper: dense view
def to_dense(m):
    return m.toarray() if issparse(m) else np.asarray(m)

# ---------------------------------------------
# 2) Aggregate per-group (tech and cell type)
# ---------------------------------------------
rows = []

# Per-technology
for tech in tech_order:
    sub = rbc_cells[rbc_cells.obs["technology"].astype(str) == tech]
    X = to_dense(sub[:, present_genes].X)
    frac = (X > 0).mean(axis=0)        # fraction expressing
    mean_expr = X.mean(axis=0)         # mean raw counts
    for g, f, m in zip(present_genes, frac, mean_expr):
        rows.append({
            "Group": tech,             # keep as the code; map later for y-axis text
            "GroupType": "Technology",
            "Gene": g,
            "Fraction": f,
            "MeanExpr": np.log1p(m)    # log1p scaling for color
        })

# Per-cell-type (across ALL technologies)
present_ct = [ct for ct in wanted_celltypes if ct in set(rbc_cells.obs[celltype_col].astype(str))]
for ct in present_ct:
    sub = rbc_cells[rbc_cells.obs[celltype_col].astype(str) == ct]
    X = to_dense(sub[:, present_genes].X)
    frac = (X > 0).mean(axis=0)
    mean_expr = X.mean(axis=0)
    for g, f, m in zip(present_genes, frac, mean_expr):
        rows.append({
            "Group": ct,               # exact cell type string; no mapping
            "GroupType": "CellType",
            "Gene": g,
            "Fraction": f,
            "MeanExpr": np.log1p(m)
        })

dot_df = pd.DataFrame(rows)

# ---------------------------------------------
# 3) Build y-axis order: techs, then cell types
# ---------------------------------------------
y_order = tech_order + present_ct
# Map tech codes → descriptive labels for y-ticks; keep cell-type names as-is
def pretty_group_label(x):
    return tech_map.get(x, x)

# Ensure categorical ordering
dot_df["Group"] = pd.Categorical(dot_df["Group"], categories=y_order, ordered=True)
dot_df["Gene"] = pd.Categorical(dot_df["Gene"], categories=present_genes, ordered=True)

# ---------------------------------------------
# 4) Scatter plot: size=fraction, color=log1p(mean)
# ---------------------------------------------
fig_h = 0.8 * len(y_order) + 2
fig_w = 1.1 * len(present_genes) + 3
fig, ax = plt.subplots(figsize=(fig_w, fig_h))

# Coordinates on grid
x_pos = {g: i for i, g in enumerate(present_genes)}
y_pos = {grp: i for i, grp in enumerate(y_order)}

xs = dot_df["Gene"].map(x_pos).values
ys = dot_df["Group"].map(y_pos).values

# Fraction → size (% legend will show 20–100)
sizes = (dot_df["Fraction"].values * 100.0)  # as percent
size_scale = 9.0  # visual multiplier (tweak if dots look too small/large)
sizes_pts = (sizes * size_scale)            # points^2 for scatter

# Color = log1p(mean expression); use white→dark red
colors = dot_df["MeanExpr"].values
sc = ax.scatter(xs, ys, s=sizes_pts, c=colors, cmap="Reds", edgecolor="none")

# Axes
ax.set_xticks(range(len(present_genes)))
ax.set_xticklabels(present_genes, rotation=45, ha="right")
ax.set_yticks(range(len(y_order)))
ax.set_yticklabels([pretty_group_label(y) for y in y_order])

ax.set_xlabel("Gene")
ax.set_ylabel("Group (Technology then Cell type)")
ax.set_title("Figure 2D — Fraction of cells (size) and mean expression (color) across groups")

# ---------------------------------------------
# 5) Legends on the right
# ---------------------------------------------
# (A) Size legend: Fraction (%) = 20, 40, 60, 80, 100
legend_percents = [20, 40, 60, 80, 100]
legend_sizes = [p * size_scale for p in legend_percents]

# Dummy handles for size legend
handles = [plt.scatter([], [], s=s, color="gray", edgecolor="none") for s in legend_sizes]
labels = [f"{p}" for p in legend_percents]

size_legend = ax.legend(
    handles, labels,
    title="Fraction of cells in group (%)",
    loc="center left",
    bbox_to_anchor=(1.02, 0.5),
    frameon=True
)
ax.add_artist(size_legend)

# (B) Colorbar: mean expression in group (log1p scale with ticks 1,2,3)
cbar = fig.colorbar(sc, ax=ax, fraction=0.046, pad=0.04)
cbar.set_label("mean expression in group")
cbar.set_ticks([1, 2, 3])
cbar.set_ticklabels(["1", "2", "3"])

plt.tight_layout()
plt.show()

# ---------------------------------------------
# 6) Notes
# ---------------------------------------------
# • Rows list all technologies first (pretty mapped names), then five immune cell types.
# • Dot size legend shows fraction (%) with reference sizes at 20/40/60/80/100.
# • Colorbar uses log1p(mean) but is *labeled* with 1/2/3 for easy reading.
# • If a row is missing, that group had no RBC-depleted cells (or cell type not present).
# • Adjust `size_scale` if dots look too large/small for your monitor or export size.


In [ ]:
# =============================================================================
# Figure 3A — Same UMAP embedding, colored by Technology / Tissue / Cell type
# =============================================================================
# Inputs:
#   adata_annot: annotated AnnData with metadata in .obs and UMAP in .obsm['X_umap']
# Required .obs columns:
#   - 'technology' (codes like 10X3prime, 10Xfixed, Parse, HIVE)
#   - 'tissue' (PBMC; RBC-depleted may be embedded in 'ID' field)
#   - one of ['celltype1','celltype0','celltype2','dblabel'] for cell identity
#
# Output:
#   Three panels with identical UMAP shapes:
#     Left:  colored by technology
#     Middle: colored by tissue (PBMC vs RBCdepl)
#     Right: colored by cell type

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# ----------------------------
# 1) Build a plotting DataFrame
# ----------------------------
if 'X_umap' not in adata_annot.obsm_keys():
    raise KeyError("UMAP coordinates not found in adata_annot.obsm['X_umap'].")

umap = adata_annot.obsm['X_umap']
df = pd.DataFrame({
    'UMAP1': umap[:, 0],
    'UMAP2': umap[:, 1],
})

# Technology codes (required)
if 'technology' not in adata_annot.obs.columns:
    raise KeyError("Expected 'technology' in adata_annot.obs.")
df['technology'] = adata_annot.obs['technology'].astype(str).values

# Tissue:
# - Use 'tissue' if present (often "PBMC")
# - Add "RBCdepl" based on 'ID' substring (case-insensitive)
tissue_series = None
if 'tissue' in adata_annot.obs.columns:
    tissue_series = adata_annot.obs['tissue'].astype(str)
else:
    tissue_series = pd.Series(['Unknown'] * adata_annot.n_obs, index=adata_annot.obs_names)

if 'ID' in adata_annot.obs.columns:
    id_str = adata_annot.obs['ID'].astype(str)
    is_rbc = id_str.str.contains('RBC', case=False, na=False)
    # Prefer explicit RBCdepl over generic tissue label
    tissue_series = np.where(is_rbc, 'RBCdepl', tissue_series)

df['tissue'] = pd.Series(tissue_series, index=adata_annot.obs_names).astype(str).values

# Cell type column: pick the best available
celltype_candidates = ['celltype1','celltype0','celltype2','dblabel']
celltype_col = next((c for c in celltype_candidates if c in adata_annot.obs.columns), None)
if celltype_col is None:
    raise KeyError("No cell type column found (looked for 'celltype1','celltype0','celltype2','dblabel').")

df['celltype'] = adata_annot.obs[celltype_col].astype(str).values

# ---------------------------------------
# 2) Category orders and nice label maps
# ---------------------------------------
tech_order = [t for t in ['10X3prime','10Xfixed','Parse','HIVE'] if t in df['technology'].unique()]
tech_map = {
    '10X3prime': "Chromium Single Cell 3′ Gene Expression v3.1 (10X Genomics)",
    '10Xfixed':  "Chromium Single Cell Gene Expression Flex (10X Genomics)",
    'Parse':     "Evercode WT Mini v2 (Parse Biosciences)",
    'HIVE':      "HIVE scRNAseq v1 (Honeycomb Biotechnologies)",
}
tech_palette = sns.color_palette('Set2', n_colors=max(1, len(tech_order)))
tech_color_map = {code: tech_palette[i] for i, code in enumerate(tech_order)}

# Tissue order & colors (PBMC vs RBCdepl)
tissue_order = [t for t in ['PBMC','RBCdepl'] if t in df['tissue'].unique()]
# Fallback: keep whatever is there, but ensure deterministic order
if not tissue_order:
    tissue_order = sorted(df['tissue'].unique().tolist())

tissue_color_map = {
    'PBMC':    '#1f77b4',  # blue
    'RBCdepl': '#d62728',  # red
}
# Fill any unknowns with gray
for t in df['tissue'].unique():
    tissue_color_map.setdefault(t, '#7f7f7f')

# Cell type order: put common immune types first if present
preferred_celltypes = [
    'B cell', 'T cell', 'Natural killer cell', 'monocyte',
    'neutrophil', 'granulocyte', 'Hematopoietic multipotent progenitor cell'
]
present_celltypes = df['celltype'].unique().tolist()
celltype_order = [c for c in preferred_celltypes if c in present_celltypes]
celltype_order += [c for c in present_celltypes if c not in celltype_order]  # append the rest

# Distinct palette for cell types (extend as needed)
celltype_colors = sns.color_palette('tab20', n_colors=len(celltype_order))
celltype_color_map = {c: celltype_colors[i] for i, c in enumerate(celltype_order)}

# ---------------------------------------
# 3) Plot — three panels, same axes limits
# ---------------------------------------
# Compute shared limits so shapes are identical
xlim = (df['UMAP1'].min() - 0.5, df['UMAP1'].max() + 0.5)
ylim = (df['UMAP2'].min() - 0.5, df['UMAP2'].max() + 0.5)

fig, axes = plt.subplots(1, 3, figsize=(18, 5), constrained_layout=True)

# Panel 1: Technology
ax = axes[0]
# map colors for each point
colors_tech = df['technology'].map(lambda t: tech_color_map.get(t, '#7f7f7f'))
ax.scatter(df['UMAP1'], df['UMAP2'], c=colors_tech, s=4, alpha=0.7, linewidths=0)
ax.set_title('Technology')
ax.set_xlim(*xlim); ax.set_ylim(*ylim)
ax.set_xticks([]); ax.set_yticks([])
# Legend (compact)
handles = [plt.Line2D([0],[0], marker='o', color='w', markerfacecolor=tech_color_map[c], markersize=8, label=tech_map.get(c,c))
           for c in tech_order]
ax.legend(handles=handles, title='', loc='lower left', fontsize=8, frameon=False)

# Panel 2: Tissue (PBMC vs RBCdepl)
ax = axes[1]
colors_tissue = df['tissue'].map(lambda t: tissue_color_map.get(t, '#7f7f7f'))
ax.scatter(df['UMAP1'], df['UMAP2'], c=colors_tissue, s=4, alpha=0.7, linewidths=0)
ax.set_title('Tissue')
ax.set_xlim(*xlim); ax.set_ylim(*ylim)
ax.set_xticks([]); ax.set_yticks([])
handles = [plt.Line2D([0],[0], marker='o', color='w', markerfacecolor=tissue_color_map[c], markersize=8, label=c)
           for c in tissue_order]
ax.legend(handles=handles, title='', loc='lower left', fontsize=8, frameon=False)

# Panel 3: Cell type
ax = axes[2]
colors_cell = df['celltype'].map(lambda c: celltype_color_map.get(c, '#7f7f7f'))
ax.scatter(df['UMAP1'], df['UMAP2'], c=colors_cell, s=4, alpha=0.7, linewidths=0)
ax.set_title('Cell type')
ax.set_xlim(*xlim); ax.set_ylim(*ylim)
ax.set_xticks([]); ax.set_yticks([])
# Put a scrollable-feel legend (may be long)
handles = [plt.Line2D([0],[0], marker='o', color='w', markerfacecolor=celltype_color_map[c], markersize=6, label=c)
           for c in celltype_order[:15]]  # show top ~15 to avoid clutter; adjust as needed
ax.legend(handles=handles, title='', loc='lower left', fontsize=7, frameon=False)

plt.suptitle('Figure 3A — Same UMAP, colored by Technology / Tissue / Cell type', y=1.02)
plt.show()


In [ ]:
adata = bc.st.clustering(adata, results_folder, myres=1.0)
adata

In [ ]:
if rbc_cells.obs.get("percent_mito") is None:
    bc.pp.fraction_counts(adata=adata)
if adata.obs.get("n_counts") is None:
    adata.obs["n_counts"] = adata.X.sum(axis=1)
if adata.obs.get("n_genes") is None:
    adata.obs["n_genes"] = np.sum(adata.X > 0, axis=1)